In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
data_dir

'./datasets/flower_photos'

In [ ]:
import pathlib
data_dir = pathlib.Path(data_dir) / 'flower_photos'
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos/flower_photos')

In [ ]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [ ]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [ ]:
IMG_WIDTH = 180
IMG_HEIGHT = 180

In [ ]:
# Removed the data_augmentation sequential model definition

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# ✅ Step 1: Prepare data lists
all_image_paths = []
all_image_labels = []

for label in flowers_images_dict:
    image_list = flowers_images_dict.get(label, [])
    label_index = flowers_labels_dict.get(label)

    for image_path in image_list:
        all_image_paths.append(str(image_path))
        all_image_labels.append(label_index)

# ✅ Step 2: Check if data was loaded correctly
print("Total image paths:", len(all_image_paths))
print("Total labels:", len(all_image_labels))

if len(all_image_paths) == 0:
    raise ValueError("No image paths found. Please check your data dictionaries.")

# ✅ Step 3: Convert to numpy arrays
all_image_paths = np.array(all_image_paths)
all_image_labels = np.array(all_image_labels)

# ✅ Step 4: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    all_image_paths,
    all_image_labels,
    test_size=0.2,
    random_state=42,
    stratify=all_image_labels  # Ensures class balance
)

# ✅ Step 5: Output confirmation
print("Training set size:", len(X_train))
print("Test set size:", len(X_test))


Total image paths: 3670
Total labels: 3670
Training set size: 2936
Test set size: 734


In [ ]:
# Removed redundant code for scaling X_train and y_train as tf.data pipeline is used for data loading and preprocessing.
# The datasets train_ds and test_ds are created and preprocessed in cell d1360778.

In [ ]:
def preprocess_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    img = img / 255.0  # Normalize to [0, 1]
    return img, label

# Create TensorFlow Datasets from original paths and labels
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Apply preprocessing
train_ds = train_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)


# Batch the datasets
train_ds = train_ds.batch(32)
test_ds = test_ds.batch(32)

# Apply data augmentation to the training dataset (assuming data_augmentation is defined elsewhere)
# If data_augmentation is part of the model, remove this line
# train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)


# Cache and prefetch the datasets
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
num_classes = 5
IMG_HEIGHT = 180
IMG_WIDTH = 180

# Build the model
model = keras.Sequential([
    keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),  # Input layer
    layers.RandomFlip("horizontal"), # Added data augmentation layers
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)), # Explicitly set input shape
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)  # Output logits
])

# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # use 'from_logits=True' for raw outputs
    metrics=['accuracy']
)


history = model.fit(
    train_ds,
    validation_data=test_ds,
    batch_size=32,
    epochs=30
)

NameError: name 'keras' is not defined

In [ ]:
# prompt: print the test accuracy

loss, accuracy = model.evaluate(test_ds)
print('Test accuracy :', accuracy)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7057 - loss: 1.0275
Test accuracy : 0.7125340700149536
